# Introduction to `Spark` RDD

In [1]:
import numpy as np

In [2]:
import os
import sys
import inspect

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName("Spark RDD Course")
sc = SparkContext(conf=conf)

In [4]:
rdd = sc.parallelize(range(64))

Note that `parallelize` takes an optional argument to choose the number
of partitions

In [5]:
rdd.getNumPartitions()

In [6]:
rdd = sc.parallelize(range(1000), 10)
rdd.getNumPartitions()

## Transformations

### `map`

In [7]:
rdd = sc.parallelize([2, 3, 4])
rdd = rdd.map(lambda x: list(range(1, x)))

In [8]:
rdd

In [9]:
(
    sc.parallelize([2, 3, 4])
      .map(lambda x: list(range(1, x)))
)

`map` is a *transformation*. It is *lazily* evaluated. Hence execution
is delayed until an *action* is met in the DAG).

In [10]:
rdd.collect()  # collect is an action 

In [11]:
(
    sc.parallelize([2, 3, 4])
      .map(lambda x: list(range(1, x)))
      .collect()
)

### Exercice: `map` with a method

**Warning.** This example is a bad practice !!! Don’t do this at home

In [12]:
dbtel = {'arthur': 1234, 'riad': 4567, 'anatole': 3615}

In [13]:
class TelephoneDB(object):
    
    def __init__(self):
        self.tel = {'arthur': 1234, 'riad': 4567, 'anatole': 3615}
   
    def add_tel(self, name):
        return name, self.tel.get(name)

In [14]:
tel_db = TelephoneDB()
names = ['arthur', 'riad']

In [15]:
rdd = (
    sc
        .parallelize(names)
        .map(tel_db.add_tel)
        .collect()
)

rdd

-   Replace the `tel` dictionary by a `defaultdict` with default number
    `999`
-   Use it on a `rdd` containing names as above including an unknown
    one, and try it

In [16]:
from collections import defaultdict

class TelephoneDefaultDB(object):
    
    def __init__(self):
        self.tel = defaultdict(lambda: 999, {'arthur': 1234, 'riad': 4567, 'anatole': 3615})
    
    def add_tel(self, name):
        return name, self.tel[name]
    
    def add_tel_rdd(self, rdd):  
        return rdd.map(self.add_tel)

In [17]:
tel_db = TelephoneDefaultDB()
names = ['riad', 'anatole', 'yiyang']
rdd = (
    sc
        .parallelize(names)
        .map(tel_db.add_tel)
        .collect()
)
rdd

> **Caution**
>
> It is a bad idea to pass *methods* to spark’s `map`. Since `add_tel`
> needs `self`, the whole object is serialized so that `spark` can use
> it.
>
> This breaks if the `tel` is large, or if it is not serializable.

### `flatMap`

In [18]:
rdd = sc.parallelize([2, 3, 4, 5])
( 
    rdd
        .flatMap(lambda x: range(1, x))
        .collect()
)

### `filter`

In [19]:
rdd = sc.parallelize(range(10))

rdd\
    .filter(lambda x: x % 2 == 0)\
    .collect()

### `distinct`

In [20]:
rdd = sc.parallelize([1, 1, 4, 2, 1, 3, 3])
rdd.distinct().collect()

### “Pseudo-set” operations

In [21]:
rdd1 = sc.parallelize(range(5))
rdd2 = sc.parallelize(range(3, 9))
rdd3 = rdd1.union(rdd2)
rdd3.collect()

In [22]:
rdd3.distinct().collect()

In [23]:
rdd1 = sc.parallelize([1, 2])
rdd2 = sc.parallelize(["a", "b"])
rdd1.cartesian(rdd2).collect()

## Actions

`collect` is obviously an action…

### `count`, `countByValue`

In [24]:
rdd = sc.parallelize([1, 3, 1, 2, 2, 2])
rdd.count()

In [25]:
rdd.countByValue()

Why does `countByValue()` returns a dictionary?

Are `count()` and `countByValue()` actions or transformations?

In [26]:
u = np.int32((np.random.sample(100000) * 100000))  # 100000 random integers uniformly distributed on 0, ..., 100000

p = (
    sc.parallelize(u)
    .countByValue()
)

q = sorted(
    p.items(), 
    key = lambda x : x[1], 
    reverse=True
)

q[0:10]

q[0], 1 + np.log(len(u))/ np.log(np.log(len(u))), len(q)

-   How many distinct values do you expect in `u` ?
-   How large is the largest value in $q$ ?

In [27]:
from scipy.stats import poisson 

( 
    len(q), 
    (1-np.exp(-1)) * len(u),
    poisson.ppf(1.-1./len(u), 1)
)

### `take`, `takeOrdered`

In [28]:
rdd = sc.parallelize([(3, 'a'), (1, 'b'), (2, 'd')])

In [29]:
(1, 'b') <=  (2, 'd') <= (3, 'a')

In [30]:
rdd.takeOrdered(2)

In [31]:
rdd.takeOrdered(2, key=lambda x: x[1])

### `reduce`, `fold`

In [32]:
rdd = sc.range(1, 4)
rdd.reduce(lambda a, b: a + b)

In [33]:
rdd = sc.range(1, 4, numSlices=7)
rdd.reduce(lambda a, b: a + b)

In [34]:
rdd = sc.parallelize(range(1,4), 3)
rdd.reduce(lambda a, b: a + b)

In [35]:
( 
    sc.parallelize(range(1, 4), 2)
      .fold(0, lambda a, b: a + b)
)

In [36]:
( 
    sc.parallelize(range(1, 4), 1)
      .fold(3, lambda a, b: a + b)
),( 
    sc.parallelize(range(1, 4), 2)
      .fold(2, lambda a, b: a + b)
)

In [37]:
rdd =  sc.parallelize(range(1, 4),3)
( 
    rdd.fold(1, lambda a, b: a + b), 
    rdd.getNumPartitions()
)

In [38]:
rdd =  sc.parallelize(range(1, 4),4)

(
    rdd.fold(1, lambda a, b: a + b), 
    rdd.getNumPartitions()
)

In [39]:
rdd = sc.parallelize([1, 2, 4], 2)
rdd.fold(2, lambda a, b: a + b)

In [40]:
rdd = sc.parallelize([1, 2, 4], 3)
rdd.fold(2, lambda a, b: a + b)

In [41]:
rdd.getNumPartitions()

### `aggregate`

In [42]:
seqOp = lambda x, y: (x[0] + y, x[1] + 1)
combOp = lambda x, y: (x[0] + y[0], x[1] + y[1])

rdd = sc.parallelize([1, 2, 3, 4], 8)
(
    rdd.aggregate((0, 0), seqOp, combOp), rdd.getNumPartitions()
)

In [43]:
op = lambda x, y: x+y
rdd = sc.parallelize([1, 2, 3, 4], 4)
(
    rdd.aggregate(0, op, op),
    rdd.getNumPartitions()
)

### Exercice: sum of powers with `aggregate`

-   Using `aggregate`, compute the sum, the sum of squares $x^2$ and the
    sum of cubes $x^3$ for $x \in \{1, \ldots, 10 \}$.
-   Check your computations using `numpy`

In [44]:
seqOp = lambda x, y: (x[0] + y, x[1] + y ** 2, x[2] + y ** 3)

In [45]:
combOp = lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2])

In [46]:
sc.range(5)

In [47]:
( 
    sc
        .range(1, 11)
        .aggregate((0, 0, 0), seqOp, combOp)
)

In [48]:
import numpy as np

x = np.arange(1, 11)
x

In [49]:
x.sum(), (x**2).sum(), (x**3).sum(), x.cumsum()

### Computing an empirical variance with `aggregate`

Assume a sample is stored as a RDD. Using `aggregate`, compute the
sample variance $\frac{1}{n}\sum_{i=1}^n (x_i - \overline{X}_n)^2$ where
$\overline{X}_n = \frac{1}{n} \sum_{i=1}^n x_i$

# `PairRDD`

In [50]:
rdd = sc.parallelize([[1, "a", 7], [2, "b", 13], [2, "c", 17]])

rdd.collect()  # not yet 

In [51]:
rdd = rdd.map(lambda x: (x[0], x[1:]))

rdd.collect()  # done 

## Transformations

### `keys`, `values`

In [52]:
rdd.keys().collect()

In [53]:
rdd.values().collect()

> **Warning**
>
> All elements must be tuples with two elements (key and value)

In [54]:
rdd = sc.parallelize([[1, "a", 7], [2, "b", 13], [2, "c", 17]])
rdd.keys().collect()

In [55]:
rdd.values().collect()

The values are *not* what we expected wrong… so we *must* do

In [56]:
rdd = ( sc.parallelize([[1, "a", 7], [2, "b", 13], [2, "c", 17]])
          .map(lambda x: (x[0], x[1:]))
      )
rdd.keys().collect()

In [57]:
rdd.values().collect()

Now, the values are correct.

### `mapValues`, `flatMapValues`

In [58]:
rdd = sc.parallelize([("a", "x y z"), ("b", "p r")])

rdd.mapValues(lambda v: v.split(' ')).collect(), rdd.collect()

In [59]:
rdd.flatMapValues(lambda v: v.split(' ')).collect()

### `groupByKey`

In [60]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1), ("b", 3), ("c", 42)])
( 
    rdd.groupByKey()
       .mapValues(list)
       .collect()
)

In [61]:
rdd.groupByKey().collect()

### `reduceByKey`

In [62]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
rdd.reduceByKey(lambda a, b: a + b).collect()

### `combineByKey`

In [63]:
rdd = sc.parallelize([('a', 1), ('b', 2), ('a', 13)])

def add(a, b): 
    return a + str(b)

rdd.combineByKey(str, add, add).collect()

### `join`, `rightOuterJoin`, `leftOuterJoin`

In [64]:
employees = sc.parallelize([
    (31, "Rafferty"),
    (33, "Jones"),
    (33, "Heisenberg"),
    (34, "Robinson"),
    (34, "Smith"),
    (None, "Williams")
])

In [65]:
departments = sc.parallelize([
    (31, "Sales"),
    (33, "Engineering"),
    (34, "Clerical"),
    (35, "Marketing")
])

In [66]:
( 
    employees
        .join(departments)
        .sortByKey()
        .collect()
)

In [67]:
( 
    employees
        .rightOuterJoin(departments)
        .sortByKey()
        .collect()
)

In [68]:
(
    employees
        .leftOuterJoin(departments)
        .collect()
)

## Actions

In [69]:
employees.countByKey()

In [70]:
employees.lookup(33)

In [71]:
employees.lookup(None)

In [72]:
employees.collectAsMap()

## References

[Spark Core
reference](https://spark.apache.org/docs/latest/rdd-programming-guide.html)